<a href="https://colab.research.google.com/github/samia-boubaya/project-01-escape-room-game/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# **PROJECT 01: ESCAPE ROOM GAME**
### IRONHACK DATA BOOTCAMP

Welcome to escape room quest

You wake up in a couch in a misteryous house with locked doors.
Your mission: Find the keys, unlock each room, and escape to freedom!

In this notebook we start by building the data structures that define the game world, which are: rooms, objects, keys and doors.

**Definition of an escape room:**

The briefing room of an escape room in Tampere, Finland
Escape rooms are inspired by escape room video games, live-action role-playing, point-and-click adventure games, puzzle hunts, interactive theater, and haunted houses. They are also referred to as "room escapes", "escape games", "exit games", or "live escapes".


---

## TEAM NOTES ⭐
### TO ADD:
*   Quiz the player when door is unlocked (comes after the unlock_door function)
*   Tell the player how much time he has left to escape and win this game (print and library of time)
*   Add how much it took the player to finish the game (important in the game_state)
*   Add Live timer and show player their countdown timer (extra)

*   Add list of game_doors to compare with current_space doors to show availabe doors for the player to after that choose which door to try unlock the door using keys from inventory.
*   horror sounds to every room, and KA-CHING for unlocking door, and different winning sound when outside

---

### ERROR HANDLING:
*   Capital letters input error when comparing with our data 
*   Capitalize first letter when printing using .title()
*   No input and enter error






--------

## EXTRA FEATURE :  GAME TRIGGERS

--------
--------
--------
--------
--------
--------
# NEW RODRIGO CODE

## Fixed code, must use this to work in Mac and Windows


In [ ]:
!pip install pillow
!pip install playsound==1.2.2
!pip install pyobjc

## ALMOST FINAL RODRIGO SOUND+IMAGE

In [ ]:
import os
import time
import threading
import platform
from PIL import Image
from IPython.display import display, clear_output
import requests
import hashlib

try:
    from playsound import playsound
except ImportError:
    raise ImportError("Please install playsound with: pip install playsound==1.2.2")

# =============================================
# ⚙️ CONFIGURATION
# =============================================

TRIGGERS = {
    "space sound": {  # Ambient background loop
        "sound": "https://drive.google.com/uc?export=download&id=1TH3ssE0tVnfKySil0q55grCoiFpCxFO7",
        "image": None,
        "loop_minutes": 20,  # 20-minute ambience
        "is_ambient": True,  # special flag
    },
    "examine": {
        "sound": "https://drive.google.com/uc?export=download&id=1_FbFeK51rtIMWg9yQovpurNGHHWukYoD",
        "image": "https://drive.google.com/uc?export=download&id=1igPXeO1Ro-UUL1tMQsMAQqOZE1kPundT",
    },
    "found key": {
        "sound": "https://drive.google.com/uc?export=download&id=1MhGR7E9YXp7LrzKaSMlCGd211I18P6Iy",
        "image": "https://drive.google.com/uc?export=download&id=1hcyiCkmmchsGZ2ERsx8PMdEN3kh--tO5",
    },
    "not found key": {
        "sound": "https://drive.google.com/uc?export=download&id=17v52HWgz_jCHFITeCV1oyOOC4TV_gyzT",
        "image": None,
    },
    "door locked": {
        "sound": "https://drive.google.com/uc?export=download&id=1-BwRmk_3U6TRESsPja5s26ECB8GYcqOU",
        "image": "https://drive.google.com/uc?export=download&id=1drL22OHO91OgjpT942g2nrmgRZcr0gYa",
    },
    "door unlocked": {
        "sound": "https://drive.google.com/uc?export=download&id=16bhcdxVRzIRmNO0ztLaeQZYHBLPVlB8P",
        "image": "https://drive.google.com/uc?export=download&id=1TXSjKZXC_K0c4gXrQlTY20u_X2aEVQ1_",
    },
    "footstep": {
        "sound": "https://drive.google.com/uc?export=download&id=1mlkfF9RW6uhKMeBqixYZlo5w-dVJ_1Uo",
        "image": "https://drive.google.com/uc?export=download&id=16snfruJnkMN8fkxL_fxDzwCaNhNdRroN",
    },
    "outside sound": {
        "sound": "https://drive.google.com/uc?export=download&id=13vxJyjMuAz0hHfoaqLIrmOEbf5IN8BGW",
        "image": None,
    },
}

CACHE_DIR = "cache"
os.makedirs(CACHE_DIR, exist_ok=True)

# =============================================
# 🧠 UTILITIES
# =============================================

def download_file(url: str, name: str) -> str | None:
    """Download a file if not cached, return local path inside assets/images or assets/sounds."""
    # Determine subfolder based on file type
    _, ext = os.path.splitext(name.lower())
    if ext in [".mp3", ".wav", ".ogg"]:
        subfolder = "sounds"
    else:
        subfolder = "images"

    # Define asset folder structure (relative to current repo)
    base_assets_dir = os.path.join(os.getcwd(), "assets")
    target_dir = os.path.join(base_assets_dir, subfolder)
    os.makedirs(target_dir, exist_ok=True)

    # Create unique hashed filename
    url_hash = hashlib.md5(url.encode()).hexdigest()[:8]
    base, ext = os.path.splitext(name)
    unique_name = f"{base}_{url_hash}{ext}"
    local_path = os.path.join(target_dir, unique_name)

    # Return cached version if it exists
    if os.path.exists(local_path):
        return local_path

    print(f"⬇️ Downloading {name} → {target_dir}/")
    try:
        response = requests.get(url, timeout=15)
        response.raise_for_status()
        with open(local_path, "wb") as f:
            f.write(response.content)
        return local_path
    except Exception as e:
        print(f"⚠️ Download error: {e}")
        return None


# =============================================
# 🔊 SOUND PLAYER
# =============================================

_event_lock = threading.Lock()  # ensures only one event sound at a time

def play_sound(url: str, trigger_name: str, loop_duration: int = None, ambient=False):
    """Play a sound (ambient loops separately, event sounds locked)."""
    local_file = download_file(url, f"{trigger_name}.mp3")
    if not local_file:
        return

    def _play():
        try:
            start = time.time()
            while True:
                # Only lock non-ambient sounds so they don’t overlap
                if not ambient:
                    with _event_lock:
                        _play_once(local_file)
                else:
                    _play_once(local_file)

                if not loop_duration or (time.time() - start > loop_duration):
                    break
        except Exception as e:
            print(f"⚠️ Sound error: {e}")

    threading.Thread(target=_play, daemon=True).start()


def _play_once(path):
    """Play one file once, platform-safe."""
    if platform.system() == "Darwin":
        os.system(f"afplay '{path}' >/dev/null 2>&1")
    elif platform.system() == "Linux":
        os.system(f"mpg123 '{path}' >/dev/null 2>&1")
    else:
        playsound(path, block=True)

# =============================================
# 🖼️ IMAGE DISPLAY (INLINE)
# =============================================

def show_image_inline(url: str, display_time: int = 5000):
    if not url:
        return
    try:
        local_file = download_file(url, "image.gif")
        if not local_file:
            return
        img = Image.open(local_file)
        img = img.resize((400, 300), Image.Resampling.LANCZOS)
        display(img)

        def safe_clear():
            try:
                clear_output(wait=True)
            except LookupError:
                pass
        threading.Timer(display_time / 1000, safe_clear).start()

    except Exception as e:
        print(f"Image error: {e}")

# =============================================
# 🚀 MAIN TRIGGER FUNCTION
# =============================================

def trigger_event(trigger_name: str, display_time: int = 5000):
    trigger_name = trigger_name.lower().strip()
    data = TRIGGERS.get(trigger_name)
    if not data:
        print(f"⚠️ Unknown trigger: {trigger_name}")
        return

    loop_duration = None
    ambient = data.get("is_ambient", False)
    if data.get("loop_minutes"):
        loop_duration = data["loop_minutes"] * 60

    # 🎧 play sound
    if data.get("sound"):
        play_sound(data["sound"], trigger_name, loop_duration, ambient)

    # 🖼️ show image
    if data.get("image"):
        show_image_inline(data["image"], display_time)


In [ ]:
# TEST CALL TRIGGER EVENT

# explore 5min per room
trigger_event("space sound") #explore 5min

# examine 5sec
trigger_event("examine") # examine
trigger_event("not found key") # not found key in examined item
trigger_event("found key") # found key in examined item

# unlock door 5sec
trigger_event("door locked") # trying to unlock a door that we don't have the key in inventory for
trigger_event("quiz") # we have a quiz to be able to unlock 
trigger_event("door unlocked") # we have key in inventory and we answered the quiz correctly

# navigate 5sec
trigger_event("footstep")

# win 5sec
trigger_event("outside sound")

------
## FINAL GAME PRODUCT

### DICT : GAME AREAS

In [ ]:
# Each key in this dictionary represents a room; each value is a list of objects, doors and keys

# Dictionary : GAME AREAS
game_areas = {
    "game room" : ['couch', 'piano', 'door A', 'key door A'],
    "bedroom 1": ['queen bed', 'door A', 'door B', 'door C', 'key door B'],
    "bedroom 2": ['double bed', 'door B', 'dresser', 'key door D', 'key door C'],
    "living room": ["dining table", 'door C', 'door D'],
    "outside": ["freedom"]
}

# List : dictionary Keys
spaces = game_areas.keys()
print("GAME AREAS Dictionary:")
print("Dictionary Keys:\n",spaces,"\n")

# Print dictionary organised
print("Dictionary:\n{   key   :   value     }")
for i in spaces:
  print(i,":",game_areas[i])

### LISTS:

In [22]:
# LIST : actions available to a player
actions = ['explore', 'examine', 'unlock door', 'navigate', 'restart', 'quit','play']
print("Player's actions in the game :",actions)

# LIST : spaces
spaces = ['game room', 'bedroom 1', 'bedroom 2', 'living room','outside']
print("\nSpaces in the game :",spaces)

# LIST : items
items = ['couch', 'piano', 'queen bed', 'double bed', 'dresser', "dining table"]
print("\nItems in the game :",items)

# LIST : doors
doors = ['door A', 'door B', 'door C', 'door D']
print("\nDoors in the game :",doors)

# LIST : keys
keys = ['key door A', 'key door B', 'key door C', 'key door D']
print("\nKeys in the game :",keys)

Player's actions in the game : ['explore', 'examine', 'unlock door', 'navigate', 'restart', 'quit', 'play']

Spaces in the game : ['game room', 'bedroom 1', 'bedroom 2', 'living room', 'outside']

Items in the game : ['couch', 'piano', 'queen bed', 'double bed', 'dresser', 'dining table']

Doors in the game : ['door A', 'door B', 'door C', 'door D']

Keys in the game : ['key door A', 'key door B', 'key door C', 'key door D']


### DICT : GAME STATE

### Import UTILS

In [20]:
import sys
# import utils.py to call functions 
from importlib import reload
import utils
reload(utils)
print("utils.py reloaded ✅")

Player is navigating to a new space!
utils.py reloaded ✅


### How to use UTILS

In [ ]:
from importlib import reload
import utils

# Reload the module (in case you changed it)
reload(utils)

# Example inputs
function_name = "my_function"
function_args = (1, 2, 3)                # positional arguments
function_kwargs = {"x": 10, "y": 20}     # keyword arguments

# Get the function object from the module
func = getattr(utils, function_name)

# Call it with args and kwargs
result = func(*function_args, **function_kwargs)

print(result)

### SHOW AVAILABLE ACTIONS + PLAYER INPUT

In [23]:
print("\nThe list of actions:",actions) # Print the list of actions


The list of actions: ['explore', 'examine', 'unlock door', 'navigate', 'restart', 'quit', 'play']


---
## GAME DATA (DICT & LIST)

---
#### SABINA COMMENTS
```
I see you don't have the code from the loop here. I would say to add that in so you can test the changes you do to the functions every time you make a change. At the moment it looks like you're focusing on all the different elements but don't have the full picture.

**So please try to make a working loop (either the one in the sample_code or one that you create with your own actions) as your next step before you work on any more details! The MVP (Minimum Viable Product) needs to come first!**

SABINA COMMENT: You can do `return action` instead of `break`. `return` automatically stops the function so because the while loop is INSIDE the function, then it will stop as well once the action is returned.

SABINA COMMENT: This value is hard-coded, so this function
will only work for opening door A and it will cause you issues down the line...

SABINA COMMENT: This would technically be a ValueError, not TypingError. Typing error is when you input a float instead of a string
but Value error is when you put "get me out of the game" instead of "quit"
```

---
## GAME INTRODUCTION

In [30]:
# Big "Time's Up" banner
print("\n" + "*"*60)
print("*****         WELCOME TO THE ESCAPE ROOM GAME!         *****")
print("*****               Made by POSH SPICE                 *****")
print("*"*60)

# Start the game by printing storyline message and initial game state
print('''
You wake up on a couch and find yourself in a strange house with no windows which you have never been to before.
You don't remember why you are here and what had happened before. 
You feel some unknown danger is approaching and you must get out of the house, NOW!
          ''')


************************************************************
*****         WELCOME TO THE ESCAPE ROOM GAME!         *****
*****               Made by POSH SPICE                 *****
************************************************************

You wake up on a couch and find yourself in a strange house with no windows which you have never been to before.
You don't remember why you are here and what had happened before. 
You feel some unknown danger is approaching and you must get out of the house, NOW!
          


## GAME SCENARIO START 

In [39]:
from importlib import reload
import utils
reload(utils)

#initialize start_space = 'game room'
start_space = 'game room'

# use getattr to call a function from utils
getattr(utils, 'update_space_path')(start_space)


Player is navigating to a new space!


KeyError: 'space path'

In [ ]:
from importlib import reload
import utils
reload(utils)

#initialize start_space = 'game room'
start_space = 'game room'

# call function player_action(player_input='play')
getattr(utils,'player_action')('play')


# use getattr to call a function from utils
getattr(utils, 'update_path')(start_space)


In [ ]:
# define function start_game()
def start_game():
    """
    Start the game
    """
    print("You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!")
    play_room(game_state["current_space"])

In [ ]:
# define function linebreak()
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

----
----
----
----

# EXTRA FEATURES

-----
## REYNOLD WORK

In [ ]:
# Keep track of the player : current space; current item to examine; and inventory of keys

# Dictionary : GAME STATE
game_state = {
    'space_path' : [], # player current space to navigate and make a space path
    'item_path' : [], # to select an item to examine for key; and make an item path
    'inventory': [], # to store found keys
    'time': "display_clock_countdown" , #library time for live timer and countdown
    'health': 100,
}

In [ ]:

# Check the updated dictionary
print(game_state)

In [ ]:
import time

# Initialize game state with player health
game_state = {
    'time': ["display_clock_countdown"],
    'space path': [],
    'item path': [],
    'inventory': [],
    'player_health': 100.0,  # Using float for percentage calculations
    'start_time': None,
    'next_deduction_time': 300  # Deduction starts after 5 minutes
}

def start_game():
    """Start the game's timer."""
    game_state['start_time'] = time.time()
    print("Game started! Timer is running.")

def update_player_health():
    """Update player health based on elapsed time."""
    if game_state['start_time'] is not None:
        elapsed_time = time.time() - game_state['start_time']

        # Perform deduction only if elapsed time is beyond next scheduled deduction time
        if elapsed_time >= game_state['next_deduction_time']:
            # Reduce health by 25 points
            game_state['player_health'] -= 25
            print(f"Health decreased by 15%! New health: {game_state['player_health']:.2f}")

            # Update the next deduction time for 2 minutes later
            game_state['next_deduction_time'] += 120

def game_loop():
    """Main game loop."""
    start_game()
    while game_state['player_health'] > 0:
        current_time = time.time()
        # Calculate the time left until the next deduction
        time_until_next_deduction = game_state['next_deduction_time'] - (current_time - game_state['start_time'])

        # Determine sleep time, making sure not to go negative
        sleep_time = max(0, time_until_next_deduction)

        # Sleep until the next deduction is expected
        if sleep_time > 0:
            time.sleep(sleep_time)

        # Update player health based on time
        update_player_health()

        # Provide feedback on current status
        print(f"Current player health: {game_state['player_health']:.2f}")
        if game_state['player_health'] <= 0:
            print("Game Over!")

# Start the game loop
if __name__ == "__main__":
    game_loop()

-----
## PAULINA: The Take a break

DEFINIG THE FUNCTION take_a_break()

**Step 1: The idea**

- When the player starts the game, we note the start time.
- Every second, we check how long the player has been playing.
- If it’s less than 40 minutes, keep playing.
- If it’s 40 minutes or more, stop the game and show this message:

TAKE A BREAK FOR MENTAL HEALTH: Breathe in, breathe out. Take some food, drink a cup of water & rest. Come back later.

- Then we wait 30 minutes before letting them play again

**Step 2: define what variables we need**

In [ ]:
from datetime import datetime, timedelta # timedelta means difference between two times

start_time = datetime.now()       # when the session started
break_until = None                # time when the break ends


**Step 3: The function**

In [ ]:
from datetime import datetime, timedelta # timedelta means difference between two times
import time

def take_a_break():
    global start_time, break_until

    now = datetime.now()

    # 1. If we are already in a break
    if break_until is not None:
        if now < break_until:
            remaining = break_until - now
            mins = int(remaining.total_seconds() // 60)
            secs = int(remaining.total_seconds() % 60)
            print(f"\n Still on break! Please wait {mins:02}:{secs:02} minutes.")
            return False
        else:
            # Break finished, reset timer
            print("\n Break finished! You can play again.\n")
            start_time = datetime.now()
            break_until = None
            return True

    # 2. If no break is active, check play time
    minutes_played = now - start_time
    if minutes_played >= timedelta(minutes=40):
        # Start break
        break_until = now + timedelta(minutes=1)
        print("""
========================================================
TAKE A BREAK FOR MENTAL HEALTH:
Breathe in, breathe out. Take some food, drink a cup of water & rest.
Come back later.
========================================================
""")
        return False
    else:
        return True




In [ ]:
from datetime import datetime, timedelta # timedelta means difference between two times
import time

# === GLOBAL VARIABLES ===
start_time = datetime.now()
break_until = None

def take_a_break():
    """Check if it's time for a break or if the player can keep playing."""
    global start_time, break_until

    now = datetime.now()

    # 1) If we are already in a break
    if break_until is not None:
        if now < break_until:
            remaining = break_until - now
            mins = int(remaining.total_seconds() // 60)
            secs = int(remaining.total_seconds() % 60)
            print(f"\nStill on break! Please wait {mins:02}:{secs:02} minutes.")
            return False
        else:
            # Break finished
            print("\nBreak finished! You can play again.\n")
            start_time = datetime.now()
            break_until = None
            return True

    # 2) If no break is active, check play time
    time_played = now - start_time

    # Change 'minutes' to 'seconds' for quick testing
    if time_played >= timedelta(minutes=40):
        break_until = now + timedelta(minutes=1)
        print("""
========================================================
TAKE A BREAK FOR MENTAL HEALTH:
Breathe in, breathe out. Take some food, drink a cup of water & rest.
Come back later.
========================================================
""")
        return False
    else:
        return True


# Hier ist a demo loop to show how take_a_break() works
if __name__ == "__main__":
    print("Demo started. Press Ctrl + C to stop.\n")

    try:
        while True:
            allowed = take_a_break()
            if allowed:
                print("Playing...")
            time.sleep(1)  # one tick per second
    except KeyboardInterrupt:
        print("\nStopped by user. Exiting cleanly.\n")
